##### [2022-DataMining]
# Ensemble
#### 최민영

In [1]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import  RobustScaler

from sklearn.pipeline import Pipeline

<frozen importlib._bootstrap>:219: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject


In [2]:
from utils import dict2tuplist
from evaluation import binary_clf_eval, skfold_cv

In [3]:
## 모델
from sklearn.neighbors import KNeighborsClassifier # KNN
from sklearn.naive_bayes import GaussianNB # Naive Bayes
from sklearn.linear_model import LogisticRegression # Logistic Regression
from sklearn.tree import DecisionTreeClassifier # Decision Tree

#### 데이터 불러오기

In [4]:
file_path = "C:/Users/wl498/Documents/dataminedataset-of-10s.csv"
spotify = pd.read_csv(file_path)

## input, target features
input_features = ['energy', 'danceability', 'acousticness', 'loudness','instrumentalness']
target = 'target'

## X, y split, train/test split
X,y = spotify[input_features], spotify[target]
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.3, random_state = 999) ## test size

In [5]:
from sklearn.ensemble import VotingClassifier, StackingClassifier

#### 개별 분류 모델 생성
개별 모델이 최적의 성능을 낼 수 있도록 최적화해주는 것이 필요하지만, 우선 하이퍼파라미터 최적화 없이 진행

In [6]:
knn = KNeighborsClassifier() 
dt = DecisionTreeClassifier()
lr = LogisticRegression()
nb = GaussianNB()

In [7]:
estimators = {'knn': knn,
 'dt': dt,
 'lr':lr,
 'nb':nb}
estimators = dict2tuplist(estimators)
estimators

[('knn', KNeighborsClassifier()),
 ('dt', DecisionTreeClassifier()),
 ('lr', LogisticRegression()),
 ('nb', GaussianNB())]

In [8]:
### 개별 모델 비교
pd.DataFrame({alg : skfold_cv(model, X = X_train, y = y_train).mean() for alg, model in estimators})

knn        dt        lr         nb
fold_ind  2.000000  2.000000  2.000000   2.000000
auc       0.840258  0.756451  0.860656   0.844636
acc       0.786513  0.756587  0.802145   0.741398
tpr       0.854369  0.754189  0.902030   0.947039
tnr       0.716987  0.759042  0.699831   0.530745
f1        0.801974  0.758225  0.821836   0.787582
ppv       0.755803  0.762419  0.754935   0.674134
npv       0.828014  0.750892  0.875102   0.907019
bacc      0.785678  0.756616  0.800931   0.738892
fpr       0.145631  0.245811  0.097970   0.052961
fnr       0.283013  0.240958  0.300169   0.469255
plr       5.913690  3.073247  9.590825  18.094301
nlr       2.542679  3.167329  2.341800   1.136077

### Voting Classifier
* 전반적으로 성능이 가장 낮은 분류기인 NB 모델을 제외
#### Hard Voting

In [9]:
hv_estimators = VotingClassifier(estimators=estimators[:3], 
                                 voting = 'hard',
                                 weights = [1,1,1]
                                 )

In [10]:
hv_ensemble = Pipeline(steps = [
    ('scaler', RobustScaler()),
    ('estimator', hv_estimators)  
                  ])

In [12]:
pd.DataFrame(skfold_cv(hv_ensemble, X= X_train, y = y_train, random_state = 3, predict_method='predict').mean()).T

fold_ind       auc       acc       tpr       tnr        f1       ppv  \
0       2.0  0.814859  0.815766  0.890553  0.739164  0.830268  0.777798   

        npv      bacc       fpr       fnr       plr      nlr  
0  0.868584  0.814859  0.109447  0.260836  8.298202  2.84937

In [13]:
pd.DataFrame({alg : skfold_cv(model, X = X_train, y = y_train, random_state = 3).mean() for alg, model in estimators}).T

fold_ind       auc       acc       tpr       tnr        f1       ppv  \
knn       2.0  0.840334  0.783834  0.859673  0.706159  0.800927  0.749913   
dt        2.0  0.761305  0.761505  0.765230  0.757699  0.764474  0.764051   
lr        2.0  0.859870  0.801921  0.901143  0.700287  0.821561  0.755106   
nb        2.0  0.845162  0.741628  0.946606  0.531639  0.787612  0.674388   

          npv      bacc       fpr       fnr        plr       nlr  
knn  0.831365  0.782916  0.140327  0.293841   6.276914  2.412204  
dt   0.759370  0.761465  0.234770  0.242301   3.295639  3.151213  
lr   0.874065  0.800715  0.098857  0.299713   9.318778  2.349772  
nb   0.906792  0.739123  0.053394  0.468361  18.540071  1.137759

#### Soft Voting

In [14]:
sv_estimators = VotingClassifier(estimators=estimators[:3], 
                                 voting = 'soft',
                                 weights = [1,1,3]
                                 )
sv_ensemble = Pipeline(steps = [
    ('scaler', RobustScaler()),
    ('estimator', sv_estimators)  
                  ])

pd.DataFrame(skfold_cv(sv_ensemble, X= X_train, y = y_train).mean()).T

fold_ind       auc       acc       tpr       tnr        f1       ppv  \
0       2.0  0.874207  0.819122  0.894535  0.741867  0.833386  0.780207   

        npv      bacc       fpr       fnr       plr       nlr  
0  0.873403  0.818201  0.105465  0.258133  8.910266  2.881384

In [15]:
pd.DataFrame(skfold_cv(hv_ensemble, X= X_train, y = y_train, predict_method='predict').mean()).T

fold_ind       auc       acc       tpr       tnr        f1      ppv  \
0       2.0  0.815714  0.816665  0.894528  0.736901  0.831713  0.77728   

        npv      bacc       fpr       fnr       plr       nlr  
0  0.871943  0.815714  0.105472  0.263099  8.508793  2.844898

### Stacking Classifier

In [16]:
stack_estimators = StackingClassifier(estimators=estimators[:3],
                                    final_estimator=estimators[-1][1],
                                    stack_method='predict_proba')

stack_ensemble = Pipeline(steps = [
    ('scaler', RobustScaler() ),
    ('estimators', stack_estimators)  
])

In [17]:
pd.DataFrame(skfold_cv(stack_ensemble, X= X_train, y = y_train).mean()).T

fold_ind       auc       acc       tpr       tnr        f1       ppv  \
0       2.0  0.875684  0.817766  0.891007  0.742754  0.831929  0.780498   

        npv     bacc       fpr       fnr       plr       nlr  
0  0.869587  0.81688  0.108993  0.257246  8.570287  2.941476